## Imports

In [1]:
from AlgoFCN import FeatureTracker

import torch
import pandas as pd
from tqdm import tqdm

import os
import glob
import numpy as np
import cv2

import numpy as np
import cv2
import matplotlib.pyplot as plt

import os
import glob

from tqdm import tqdm

/home/kaveh/miniconda3/envs/pt/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Run Tracker

In [3]:
address = './CNT_Growth_Dataset/'
image_paths = glob.glob(address +  '*.png')
image_paths.sort()

ft_method = 'aliked'
tracker = FeatureTracker(ftExtractor = ft_method, mx_keypoints = 4096, desired_device = 0)
kp_lent_ornt_deltaXY_dic = {}
plottingFrames = []
desRec = [0, 0, 1024, 500] # CNT expertiments
# desRec = [0, 0, 100000, 100000]
# desRec = [0, 0, 1920, 250]
# desRec = [0, 0, 1080, 185]

# desRec, _ = tracker.crop_points(202., 202., 400)
# (202, 202), (612, 500), (804, 254), (312, 700)

frameTracksList = {}
idmmf = 0
for idx, imagePath in tqdm(enumerate(image_paths)):
        frameTracks = None
        tracks, kp_lent_ornt_deltaXY_list, image_cv2, _frmTracks, frameGrowthingTracks = tracker.process_frame(imagePath, idx, desRec, isOnline = False)
        cvFrame = tracker.update_cvFrame(idx, tracks, image_cv2, _frmTracks, desRec, _isOnline = False)
        plottingFrames.append(cvFrame)
        kp_lent_ornt_deltaXY_dic[idx] = kp_lent_ornt_deltaXY_list
        # print(len(tracks), len(frameGrowthingTracks), print(idmmf))
        frameTracksList[idmmf] = frameGrowthingTracks
        idmmf = idmmf + 1

cuda:0


100it [00:18,  5.48it/s]


### Remove small Tracks

In [14]:
for key, value in list(tracks.items()):
    if len(value) < 3:
        del tracks[key]

### Save Video of Tracked CNT

In [13]:
import cv2

fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'mp4v' for .mp4 format

# Use the shape of the first image to get the frame size (width, height)
frame_size = (plottingFrames[0][0].shape[1], plottingFrames[0][0].shape[0])

saveAdrs = './results/'
output_dir = saveAdrs + ft_method + '_' + ft_method + '_cnt_tracks.mp4'
# Create a VideoWriter object. Adjust FPS to your liking.
out = cv2.VideoWriter(output_dir, fourcc, 5.0, frame_size)

for i in range(len(plottingFrames)):
    # Create a copy of the current frame to avoid modifying the original frame
    frame_with_number = plottingFrames[i][0].copy()

    # Define the text to be displayed (frame number)
    text = f"Frame: {i+1}, Name: {image_names[i]}"  # Add 1 because frame indices start at 0

    # Define the font and font scale to be used
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1

    # Define the text color and the thickness of the text
    text_color = (255, 255, 255)  # White
    thickness = 2

    # Define the position of the text
    text_position = (10, 880 - 30)  # 10 pixels from the top and 10 pixels from the left

    # Draw the text onto the frame
    cv2.putText(frame_with_number, text, text_position, font, font_scale, text_color, thickness)

    # Write the frame to the video
    out.write(frame_with_number)

# Release everything when job is finished
out.release()

# Optionally, display a message
print("The video was successfully saved!")

The video was successfully saved!
